In [1]:
import pandas as pd
from utility_functions import get_acc, add_row_to_csv

In [2]:
dataset = 'systematic'
model = 'gru'

In [3]:
cv_df = pd.read_csv(f"model/RNN/report_{dataset}_{model}.csv")
evaluation_df = pd.read_csv('training_data/' + dataset + '/evaluation.csv')
fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')

In [4]:
for test_fold in sorted(fold_df['fold'].unique()):
    df_fold = cv_df[cv_df['test_fold'] == test_fold]
    
    eval_df = evaluation_df[evaluation_df['sequenceID'].isin(fold_df[fold_df['fold'] == test_fold]['sequenceID'])]
    chosen_row = df_fold.loc[df_fold['val_loss'].idxmin()]
    n_layer = chosen_row['num_layers']
    h_size = chosen_row['hidden_size']
    pred_df = pd.read_csv(f'model/RNN/predictions/{model}_{dataset}_{n_layer}layers_{h_size}features_fold{test_fold}.csv')
    pred_df.fillna(0, inplace=True)
    acc = get_acc(eval_df, pred_df)
    add_row_to_csv('acc_rate_csvs/' + dataset + '.csv', ['method', 'fold', 'acc'], [model, test_fold, acc])